**Group member:** Jin YuHan Burgess, Sabrina Harris, Logan LaPace, Randy Otoo

**Author:** Jin YuHan Burgess

**Project Objective** 
The Student Registration System is designed to manage and prioritize student registration based on their major, year, and registration day. It employs an AVL (Adelson-Velsky and Landis) tree data structure to efficiently manage student registration and maintain a balanced order. This system can handle all students who identify as graduates, undergraduates, and auditors. The code will then return a panda dataframe of the admitted 25 students. 

**README.md**
https://github.com/JinBurgess/AlogorithmsProjectStudentCourseRegistration/blob/main/README.md

**Design Choices**
General: given the project, I assumed that this project did not allow the use of the built-in sorted function. 

A max heap which was used in previous iteration used a max-heap strucuture which had a worst case run time of O(n log n), nsertion and deletion in a max heap is O(log n), and retrieving the max or min has a time complexity of O(c).

We know that the fastest worst case rune time of a sorting algorithm is O(n log n). Given that we had already had an almost complete max-heap and wanted to do a different type of algoritm with a similar runtime. That is why we chose an AVL-Tree.

An AVL_Tree is a self-balancing binary tree. As new nodes are inserted and deleted, the tree updates it structure such that the height of the left subtree and right subtree are similar. Thus, the difference is between -1 and 1. This tree rebalancing ensures that different operations such as inserting, deletion, and look up runs in worst-case(log n).

Another aspect what was used better in this project was the class system such that it is more object-oriented.

We maintained the priority queue done from Sabrina's homework code in inserting student.

*Caveat to AVL_Tree:*
This was used as a gymnastic mental exercise to make an AVL_Tree work for a priority queue. Overall, a heap structure would be better for this situation because it uses less meemory, it is compentitionally less complicated, and runtime of an unbalanced heap structure still runs in O(n log n).


**Implementation**
The python language is an object-orientated programming language which relies on classes and objects to simplify code and reuse code chuncks. There are three classes Node, AVL_Tree, and stud_registration. The Node class defines the attributes of a node instance. The AVL_Tree contains all the functions related to maintaining the AVL structure and method to update strucutre when new students are added. The stud_registration acts like the middle man between the actions below code: if __name__ == "__main__": and the AVL_Tree class and Node class. 

**Testing and Code**

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

# class Node deals with the information about the relation and information that each node contains 
class Node:
    def __init__(self, name, major, year, reg_day, priority):
        self.name = name # student's name
        self.major = major # major of the student
        self.year = year # the year the student is in
        self.reg_day = reg_day # day of registration 
        self.priority = priority # used in comparsion based analysis for developing priority 
        self.identifer = None
        # when inputing a node there is no child attached 
        self.left = None 
        self.right = None
        self.height = 1 # will be used to help determine rebalancing 

# class AVL_Tree contains information about the structure of AVL_Tree and how to maintain it
# such as the rotation, insertion, and deletion form the tree
class AVL_Tree:
    def __init__(self):
        self.root = None # initially, the tree is empty because we have not put in any new students 

    # determines the tree height of each subtree, helps to determine if the subtree needs to be rotated and in which direction
    def tree_height(self, node):
        if node is None:
            return 0
        left_height = self.tree_height(node.left) # getting the height of the left subtree
        right_height = self.tree_height(node.right) # getting the height of the right subtree
        return max(left_height, right_height) + 1 # return the subtree which has the largest depth (adding 1 since we start our count at 0)
    
    def left_left_rotation(self, node):
        if node is not None and node.right is not None:
            new_parent = node.right # setting the right child as the pivot/new parent
            temp_var = new_parent.left # temparary variable that will be reattached when tree has been rotated
            new_parent.left = node # setting parent node as left child
            node.right = temp_var # since the node is going to be smaller than the temp var, it is going to be placed as right child

            # since we repositioned the parent node and the right child's position, we need to update the height of each
            node.height = max(self.tree_height(node.left), self.tree_height(node.right)) + 1 
            new_parent.height = max(self.tree_height(new_parent.left), self.tree_height(new_parent.right)) + 1

            # return the new_parent for recalculation to see if the tree is now balanced
            return new_parent
        else:
            return node
        
    def right_right_rotation(self, node):
        if node is not None and node.left is not None:
            new_parent = node.left # setting the left child as the pivot/new parent
            temp_var = new_parent.right # temparary variable that will be reattached when tree has been rotated
            new_parent.right = node # setting parent node as right child
            node.left = temp_var # since the node is going to be bigger than the temp var, it is going to be placed as left child

            # since we repositioned the parent node and the right child's position, we need to update the height of each
            node.height = max(self.tree_height(node.left), self.tree_height(node.right)) + 1
            new_parent.height = max(self.tree_height(new_parent.left), self.tree_height(new_parent.right)) + 1

            # return the new_parent for recalculation to see if the tree is now balanced
            return new_parent
        else:
            return node

    def left_right_rotation(self, node):
        node.left = self.left_left_rotation(node.left)
        return self.right_right_rotation(node)

    def right_left_rotation(self, node):
        node.right = self.right_right_rotation(node.right)
        return self.left_left_rotation(node)
 
    # inserting a new student into the AVL tree
    def insert(self, parent_node, name, major, year, reg_day, priority):
        # set the student to the root node if it is going to be ther first thing in the list 
        if parent_node is None:
            # in here we care using the node class to create the root node
            return Node(name, major, year, reg_day, priority)  

        # We are inserting the following students based on their priority value
        # the new student has a lower priority value than the students already in the tree
        if priority < parent_node.priority:
            parent_node.left = self.insert(parent_node.left, name, major, year, reg_day, priority)
            
        # the new student has a higher priority value than the students already in the tree
        else:
            parent_node.right = self.insert(parent_node.right, name, major, year, reg_day, priority)

        # once we add the new node we need to update the height of our tree that will be used to determine 
        # balance of our new tree
        parent_node.height = max(self.tree_height(parent_node.left), self.tree_height(parent_node.right)) + 1
       
        # if the balance is greater than 1 or smaller than -1 (so 2 or -2) means we are no longer balanced 
        # thus we need to rotate the nodes to rebalance the tree such that it is between -1>= x =<1
        balance = self.tree_height(parent_node.left) - self.tree_height(parent_node.right)
        
        # there is a string of left child that outweighs how many right child there is in the subtree
        if balance > 1:
            if priority < parent_node.left.priority:
                return self.right_right_rotation(parent_node)
            else:
                return self.left_right_rotation(parent_node)
            
        # there is a string of right child that outweighs how many left child there is in the subtree
        if balance < -1:
            if priority > parent_node.right.priority:
                return self.left_left_rotation(parent_node)
            else:
                return self.right_left_rotation(parent_node)

        return parent_node
    
    # finding the lowest priority node in a given subarray that will be used in 
    # the delete_node function
    def find_min(self, node):
        if node is None:
            return None
        
        # only looking for left child node since they will contain the smaller priority
        while node.left is not None:
            node= node.left
        return node
    
    # deleting students from tree
    def delete_node(self, node, identifier):
        if node is None: # making sure nodes exists for traversal
            return node
        if identifier < node.identifier: # going through left subtree if identifier is less than identifier of the parent node
            node.left = self.delete_node(node.left, identifier)
        elif identifier > node.identifier:
            node.right = self.delete_node(node.right, identifier) # going through right subtree if identifier is greater than identifier of the parent node
        else: # student that we want to delete is found
            # looking to see where the child exist to see which node needs to be swapped with the one that is being removed 
            if node.left is None: 
                return node.right
            elif node.right is None:
                return node.left
            temp = self.find_min(node.right) # getting the left most child of the right child and putting it into its position
            # swaping the two nodes
            node.name, node.major, node.year, node.reg_day, node.priority, node.identifier = temp.name, temp.major, temp.year, temp.reg_day, temp.priority, temp.identifier
            node.right = self.delete_node(node.right, temp.identifier) # removing the node
            node.height = 1 + max(self.tree_height(node.left), self.tree_height(node.right)) # adjusting hight
        return node
    
    # setting unique identifier to nodes so we can traverse tree to delete
    def set_identifier(self, node, N, name_to_identifier):
        if node:
            N= self.set_identifier(node.right, N, name_to_identifier) # getting higest priority student 
            node.identifier = N # setting N which is number of nodes in tree to the node
            name_to_identifier[node.name] = N # adding the asssocation between name and identifer to the dictionary
            N = N - 1 
            N = self.set_identifier(node.left, N, name_to_identifier) # getting lower priority student 
        return N

    # appending into a list, the students from the avl_ tree in descending order
    def print_tree(self, node, admitted_students, max_students):

        if node and len(admitted_students) < max_students: # making sure node exists and that we have not reached limit of list
                self.print_tree(node.right, admitted_students,max_students) # getting the right most child (student with higher priority)
                if len(admitted_students) < max_students: # making sure that we are not at limit of list

                    # appending the information of the student node to the admitted list 
                    admitted_students.append([node.name, node.major, 
                                        node.year, 
                                        node.reg_day, 
                                        node.priority,
                                        ])
                self.print_tree(node.left, admitted_students,max_students) # getting the right most child (student with lesser priority)


class stud_registration:
    def __init__(self):
        self.registration_list = AVL_Tree() # prioritization structure of students

        
    # updates the student_dict when a new student is added to the tree


# Student Priority Code
# Code by Sabrina Harris
################################################################################################################################################
    def student_priority(self, major, year, reg_day):
        level = 0 #reset the level for each function call
        if year == 0: #for auditors
            level = level + (.5-(.1*(reg_day))) # level will equal 0.5 for registrations before orientation week, 0.4 for registrations on the first day, etc.
          
        elif year <5: # for undergraduate students
            if major == 'cs' or major == 'math' or major == 'ds':
                level = level+10 # adds 10 to the level for cs or math majors,
            level = level + year # adds 1,2,3, or 4 to the level based on the year.
            level = level +(.5-(.1*(reg_day))) #+0.5 for students registered before the orientation week, same as for auditors.
            
        else:
            level = 15 #for graduate students, level will automatically be 15.
        return level
    
################################################################################################################################################

    # adding new student to the AVL tree so long as they registered within the orientation period
    def register_student(self, name, year, reg_day, major):
        if int(reg_day) < 6:
            priority = self.student_priority(major, year, reg_day) # caluclte their priority
            self.registration_list.root = self.registration_list.insert(self.registration_list.root, name, major, year, reg_day, priority)
            
    # we are removing students based on the dictionary list. I am not updating tree because
    # I am running under the assumption that the class has already started and we already of a waitlist which is captured 
    # in the dictionary
    def remove_student(self, identifier):
        self.registration_list.root = self.registration_list.delete_node(self.registration_list.root, identifier)

    # creates a dataframe that returns the 25 admitted students 
    def get_student_dataframe(self, max_students = 25):
        admitted_students = [] # list of admitted students 
        # calls this function to iterate through AVL tree and list students in descending order
        self.registration_list.print_tree(self.registration_list.root, admitted_students, max_students)

        # getting column for each data frame
        columns = ['Name', 'Major', 'Year', 'Reg Day', 'Priority'] # column names

        # return dataframe 
        return pd.DataFrame(admitted_students, columns=columns)
    
    
# Code by Jin YuHan Burgess

**Inserting Code Testing**
This line of code creates a list of 50 random students to be inserting into the tree. It then returns the 25 students who will be admitted in a dataframe. 

In [6]:
if __name__ == "__main__":
    reg_stud = stud_registration()

    random.seed(101) # set.seed for testing 

    # List of possible majors
    majors = ['cs', 'math', 'other', 'ds']

    # Function to generate a random student
    def generate_random_student(indx):
        name = f"student{indx}" # Random student name
        year = random.randint(1, 6) # Random year (greater than 0)
        if year == 0:
            major = 'none' # these are auditors
        elif year > 4:
            major = 'ds' # data science is the major so those in graduate program are on this track
        else:
            major = random.choice(majors) # Random major
        reg_day = random.randint(0, 10) # Random registration day (greater than or equal to 0)

        return name, year, reg_day, major


    # Generate a list of random students
    num_students = 50 # Change this to the number of students you want
    random_students = [generate_random_student(indx) for indx in range(num_students)] # under score in for_ in... represents that the loop variable is not needed


    # Print the generated students
    for student in random_students:
        reg_stud.register_student(student[0], student[1], student[2], student[3])
    
    name_to_identifier = {} # creating dictionary between unique identifier and student name (unique identifier is based on the location it is within the avl_tree)

    # once all the insertions are done, we set unique identifiers to each student. Students with highest priority are considered right most child and will have a higher number
    # identification and vice versa for lowest priority students 
    x = reg_stud.registration_list.set_identifier(reg_stud.registration_list.root, int(num_students), name_to_identifier) 


    admitted = reg_stud.get_student_dataframe()
    print(admitted)

# Code by Jin YuHan Burgess


         Name Major  Year  Reg Day  Priority
0   student43    ds     6        5      15.0
1   student37    ds     6        2      15.0
2   student36    ds     6        2      15.0
3   student30    ds     5        4      15.0
4   student27    ds     6        5      15.0
5   student16    ds     6        4      15.0
6   student15    ds     6        5      15.0
7    student4    ds     5        3      15.0
8    student3    ds     5        3      15.0
9    student1    ds     5        5      15.0
10   student0    ds     5        3      15.0
11  student17    ds     4        3      14.2
12  student11    ds     4        3      14.2
13  student48  math     3        0      13.5
14  student47  math     3        1      13.4
15  student40    ds     3        1      13.4
16  student33    ds     3        1      13.4
17  student13  math     3        1      13.4
18   student6    ds     3        1      13.4
19   student7  math     3        2      13.3
20  student45    ds     3        3      13.2
21   stude

**Valid Entries Code Testing**

This line of code clearly shows students who register within the orientation are admitted while those who do not sign up within the orientation period are not admitted. I set both students as the highest priority of Graduate student where one registered within the orientation while the other did not. 

In [11]:
reg_stud.register_student('studentA', 6, 6,'ds') 
reg_stud.register_student('studentB', 6, 0,'ds')

admitted = reg_stud.get_student_dataframe()
print(admitted)

# Code by Jin YuHan Burgess

         Name Major  Year  Reg Day  Priority
0    studentB    ds     6        0      15.0
1   student43    ds     6        5      15.0
2   student37    ds     6        2      15.0
3   student36    ds     6        2      15.0
4   student30    ds     5        4      15.0
5   student27    ds     6        5      15.0
6   student16    ds     6        4      15.0
7   student15    ds     6        5      15.0
8    student4    ds     5        3      15.0
9    student3    ds     5        3      15.0
10   student1    ds     5        5      15.0
11   student0    ds     5        3      15.0
12  student17    ds     4        3      14.2
13  student11    ds     4        3      14.2
14  student48  math     3        0      13.5
15  student40    ds     3        1      13.4
16  student33    ds     3        1      13.4
17  student13  math     3        1      13.4
18   student6    ds     3        1      13.4
19   student7  math     3        2      13.3
20  student45    ds     3        3      13.2
21  studen

**Deletion Code Testing**

This code is used as a reference to original dataframe minus the testing done in Valid Entries Code Testing. 

In [8]:
import pandas as pd
if __name__ == "__main__":
    reg_stud = stud_registration()

    random.seed(101) # set.seed for testing 

    # List of possible majors
    majors = ['cs', 'math', 'other', 'ds']

    # Function to generate a random student
    def generate_random_student(indx):
        name = f"student{indx}" # Random student name
        year = random.randint(1, 6) # Random year (greater than 0)
        if year == 0:
            major = 'none' # these are auditors
        elif year > 4:
            major = 'ds' # data science is the major so those in graduate program are on this track
        else:
            major = random.choice(majors) # Random major
        reg_day = random.randint(0, 10) # Random registration day (greater than or equal to 0)

        return name, year, reg_day, major

# Generate a list of random students
    num_students = 50 # Change this to the number of students you want
    random_students = [generate_random_student(indx) for indx in range(num_students)] # under score in for_ in... represents that the loop variable is not needed


    # Print the generated students
    for student in random_students:
        reg_stud.register_student(student[0], student[1], student[2], student[3])

    name_to_identifier = {} # creating dictionary between unique identifier and student name (unique identifier is based on the location it is within the avl_tree)

    # once all the insertions are done, we set unique identifiers to each student. Students with highest priority are considered right most child and will have a higher number
    # identification and vice versa for lowest priority students 
    x = reg_stud.registration_list.set_identifier(reg_stud.registration_list.root, int(num_students), name_to_identifier) 


    admitted = reg_stud.get_student_dataframe()
    print(admitted)
    
    # Code by Jin YuHan Burgess

         Name Major  Year  Reg Day  Priority
0   student43    ds     6        5      15.0
1   student37    ds     6        2      15.0
2   student36    ds     6        2      15.0
3   student30    ds     5        4      15.0
4   student27    ds     6        5      15.0
5   student16    ds     6        4      15.0
6   student15    ds     6        5      15.0
7    student4    ds     5        3      15.0
8    student3    ds     5        3      15.0
9    student1    ds     5        5      15.0
10   student0    ds     5        3      15.0
11  student17    ds     4        3      14.2
12  student11    ds     4        3      14.2
13  student48  math     3        0      13.5
14  student47  math     3        1      13.4
15  student40    ds     3        1      13.4
16  student33    ds     3        1      13.4
17  student13  math     3        1      13.4
18   student6    ds     3        1      13.4
19   student7  math     3        2      13.3
20  student45    ds     3        3      13.2
21   stude

This block sets a random sample that we can pull from to removed admitted students.

In [9]:
random.seed(101)

x = admitted.sample(n = 3, replace = False)
print(x)

# Code by Jin YuHan Burgess

         Name Major  Year  Reg Day  Priority
14  student47  math     3        1      13.4
24  student21    cs     2        5      12.0
21   student5    ds     3        3      13.2


This block of code returns an updated dataframe of students who dropped the course within the 2 week period.

In [10]:

random.seed(101)
for indx in range(len(x)):
    stud = x._get_value(indx, 0, takeable = True) # gets student name from the sample 
    identifier = name_to_identifier[stud] # finds the identifer that is attached to the student
    reg_stud.remove_student(identifier) # remove student based of their identifier
            
# return updated dataframe
admitted = reg_stud.get_student_dataframe()
print(admitted)

# Code by Jin YuHan Burgess

         Name  Major  Year  Reg Day  Priority
0   student43     ds     6        5      15.0
1   student37     ds     6        2      15.0
2   student36     ds     6        2      15.0
3   student30     ds     5        4      15.0
4   student27     ds     6        5      15.0
5   student16     ds     6        4      15.0
6   student15     ds     6        5      15.0
7    student4     ds     5        3      15.0
8    student3     ds     5        3      15.0
9    student1     ds     5        5      15.0
10   student0     ds     5        3      15.0
11  student17     ds     4        3      14.2
12  student11     ds     4        3      14.2
13  student48   math     3        0      13.5
14  student40     ds     3        1      13.4
15  student33     ds     3        1      13.4
16  student13   math     3        1      13.4
17   student6     ds     3        1      13.4
18   student7   math     3        2      13.3
19  student45     ds     3        3      13.2
20  student31     cs     2        

**Project Hurdle**
The code uses student priority to insert student into the tree. However, there is a possibility of dublicate priority. When this happens, the code  cannot iterate through the tree via the priority. The code will not be able to determine which direction to go left or right when priorities are the same but the students are not the same. Because the student name is not a valid method of iteration, we need to determine another way to get the location of the student within the tree. There needs to be a different unique identifier that is ordered in such that higher priority students have a higher identity number and lower priority students have a lower identity number. After we assign these numberse we can delete from the tree.

*Caveat* 
The order of insertion in relation to deletion needs to be such that all insertions are done betore deletions. 

Insert all students

Assign an identifier to all students 
Identifier = # node in tree
Higher priority = higher # identifier
Lower priority = lower # identifier

Delete based on identifier

I assume in this project, this structure is okay becuase delection is done once no more students can sign up for the course, and the course has already stated. If more students signed up then they would not be admitted since the did not me critera set by registration day. 


**Future extensions of the project**
I would change the priority queue to be more strict and add other variables in calculaations to remove possible duplication of priority. That way the prioirty will be distinct and the identifier added in this code will node need to be used. A vairable that I could add is a time stamp that has a precision in milliseconds. 